## Naive Bayes

formatting

In [1]:
import re
import string

def remove_punctuation(s):
    """See: http://stackoverflow.com/a/266162
    """
    exclude = set(string.punctuation)
    return ''.join(ch for ch in s if ch not in exclude)

def tokenize(text):
    text = remove_punctuation(text)
    text = text.lower()
    return re.split("\W+", text)

def count_words(words):
    wc = {}
    for word in words:
        wc[word] = wc.get(word, 0.0) + 1.0
    return wc

In [2]:
# test

s = "Hello my name, is Greg. My favorite food is pizza."
count_words(tokenize(s))

{'favorite': 1.0,
 'food': 1.0,
 'greg': 1.0,
 'hello': 1.0,
 'is': 2.0,
 'my': 2.0,
 'name': 1.0,
 'pizza': 1.0}

get corpus in place

In [19]:
from sh import find

vocab = {}
word_counts = {
    "billy_joel": {},
    "joni_mitchell": {}
}
priors = {
    "billy_joel": 0.,
    "joni_mitchell": 0.
}

docs = []
for f in find("nb_data"):
    
    f = f.strip()
    if not f.endswith(".txt"):
        # skip non .txt files
        continue
    elif "bj" in f:
        category = "billy_joel"
        #print "bj"
    else:
        category = "joni_mitchell"
        #print "jm"
    docs.append((category, f))
    # ok time to start counting stuff...
    priors[category] += 1
    text = open(f).read()
    words = tokenize(text)
    counts = count_words(words)
    for word, count in list(counts.items()):
        # if we haven't seen a word yet, let's add it to our dictionaries with a count of 0
        if word not in vocab:
            vocab[word] = 0.0  # use 0.0 here so Python does "correct" math
        if word not in word_counts[category]:
            word_counts[category][word] = 0.0
        vocab[word] += count
        word_counts[category][word] += count


test on new file

In [20]:
import math

new_doc = open("test_bj_file.txt").read()
words = tokenize(new_doc)
counts = count_words(words)

# Bayes theorem: P(A|B) = P(B|A) * (P(A)/P(B)), this is P(A)
prior_bj = (priors["billy_joel"] / sum(priors.values()))
prior_jm = (priors["joni_mitchell"] / sum(priors.values()))

prob_bj = 1.0
prob_jm = 1.0

for w, cnt in list(counts.items()):
    # skip words that we haven't seen before, or words less than 3 letters long
    if w not in vocab or len(w) <= 3:
        continue

    p_word = vocab[w] / sum(vocab.values())
    p_w_given_bj = word_counts["billy_joel"].get(w, 0.0) / sum(word_counts["billy_joel"].values())
    p_w_given_jm = word_counts["joni_mitchell"].get(w, 0.0) / sum(word_counts["joni_mitchell"].values())

    if p_w_given_bj > 0:
        prob_bj = prob_bj * (cnt * p_w_given_bj / p_word)
    if p_w_given_jm > 0:
        prob_jm = prob_jm * (cnt * p_w_given_jm / p_word)

print "Billy Joel score: ", prob_bj * prior_bj
print "Joni Mitchell score: ", prob_jm * prior_jm

Billy Joel score:  95887.6921166
Joni Mitchell score:  1.85798794576e+11
